In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import scipy.sparse as sp
import numpy as np

In [15]:
brews_df = pd.read_pickle('data/brews_with_costs.pkl')
ingredients_df = pd.read_pickle('data/ingredients.pkl')
brews_df

,first_ingredient,second_ingredient,third_ingredient,effects,name,value,descriptions
0,Abecean Longfin,Ancestor Moth Wing,Cyrodilic Spadetail,"[Fortify Restoration, Damage Stamina]",Potion of Fortify Restoration,161,"[Drain the target's Stamina by 18 points., Res..."
1,Abecean Longfin,Ash Creep Cluster,Beehive Husk,"[Fortify Sneak, Fortify Destruction]",Potion of Fortify Destruction,269,[Destruction spells are 30% stronger for 60 se...
2,Abecean Longfin,Ash Creep Cluster,Chaurus Eggs,"[Weakness to Poison, Invisibility]",Potion of Invisibility,312,"[Invisibility for 24 seconds., Target is 12% w..."
3,Abecean Longfin,Ash Creep Cluster,Cyrodilic Spadetail,"[Fortify Restoration, Damage Stamina]",Potion of Fortify Restoration,161,"[Drain the target's Stamina by 18 points., Res..."
4,Abecean Longfin,Ash Creep Cluster,Elves Ear,"[Weakness to Frost, Resist Fire]",Potion of Resist Fire,126,"[Resist 18% of fire damage for 60 seconds., Ta..."
...,...,...,...,...,...,...,...
25237,Tundra Cotton,Void Salts,NA,"[Resist Magic, Fortify Magicka]",Potion of Fortify Magicka,122,[Magicka is increased by 24 points for 60 seco...
25238,Tundra Cotton,Wisp Wrappings,NA,[Resist Magic],Potion of Resist Magic,51,[Resist 6% of magic for 60 seconds.]
25239,Vampire Dust,White Cap,NA,[Restore Magicka],Potion of Restore Magicka,25,[Restore 30 points of Magicka.]
25240,Void Salts,Wisp Wrappings,NA,[Resist Magic],Potion of Resist Magic,51,[Resist 6% of magic for 60 seconds.]


In [3]:
brews = list(brews_df.name)
ingredients = list(ingredients_df.name)

The model is the following:

#### Parameters

$B = \{1, \dots, 25242\}$: Set of brews

$I = \{1, \dots, 109\}$: Set of ingredients

$I_p \subset{I}$: Set of ingredients required to make one unit of brew $b$ (note that $|I_b| \leq 3 \,\,\forall b \in B$)

$z_i \in \mathbb{Z}^+$: Amount of ingredient $i$ available

$c_b \in \mathbb{R}^+$: gold value of brew $b$

#### Decision Variables

$y_b \in \mathbb{Z}^+$: Number of brews $b$ created

$x_{ib} \in \mathbb{Z}^+$: Amount of ingredient $i$ used to create brew $b$

#### Model

$$ \max \sum_{i \in b} c_b y_p$$
such that:
$$y_b \leq x_{ib} \qquad \forall i \in I_b,  \forall b \in B$$
$$\sum_{\{b \mid i \in I_b\}} x_{ib} \leq z_i \qquad \forall i \in I$$


An additional sophistication for the model is to allow the purchase of common ingredients, which would amount to adding a integer decision variable for each such ingredient, adding them to the RHSs of the second constraints, and subtracting the costs in the objective.

In [17]:
B = [i for i in range(len(brews_df))]
I = [i for i in range(len(ingredients_df))]
c = np.array(brews_df['value'])

In [10]:
A = [[] for b in B]
for b in B:
    brew_ingredients = brews_df.loc[b][['first_ingredient','second_ingredient','third_ingredient']]
    for ingredient in brew_ingredients:
        if ingredient != 'NA':
            i = ingredients.index(ingredient)
            A[b].append(i)
            
# randomly generate inventory
z = np.random.poisson(lam=2, size=len(I))

Using Gurobi:

In [18]:
l = gp.tuplelist()
for b in B:
    for i in A[b]:
        l.append((b,i))
        
m = gp.Model('alchemy')
x = m.addVars(l, vtype=GRB.CONTINUOUS, name='x')
y = m.addVars(B, vtype=GRB.INTEGER, name='y')
m.addConstrs((y[b] <= x[b,i] for (b,i) in l), 'c1')
m.addConstrs((x.sum('*', i) <= z[i] for i in I))

m.setObjective(gp.quicksum(c[b]*y[b] for b in B), GRB.MAXIMIZE)
m.setParam('OUTPUT_FLAG',False)
m.optimize()

In [19]:
optimal_brews = [b for b in B if y[b].x > 0.5]
optimal_df = brews_df.loc[optimal_brews]
optimal_df['count'] = [int(y[b].x) for b in B if y[b].x > 0.5]
optimal_df.sort_values(by=['value'], ascending=False)
optimal_df = optimal_df[['name', 'count', 'value', 'first_ingredient', 'second_ingredient', 'third_ingredient', 'descriptions']]
optimal_df['value_count'] = optimal_df['value'] * optimal_df['count']
optimal_df = optimal_df.sort_values(by='value_count', ascending=False)
optimal_df

,name,count,value,first_ingredient,second_ingredient,third_ingredient,descriptions,value_count
11281,Potion of Waterbreathing,1,24055,Chicken's Egg,Hawk's Egg,Salmon Roe,[Decrease the target's Magicka regeneration by...,24055
7183,Potion of Fortify Stamina,2,4023,Boar Tusk,Large Antlers,Torchbug Thorax,[Stamina is increased by 30 points for 300 sec...,8046
7126,Potion of Fortify Stamina,1,4116,Boar Tusk,Garlic,Slaughterfish Egg,[Stamina is increased by 30 points for 300 sec...,4116
18939,Potion of Fortify Health,1,3029,Giant's Toe,Glowing Mushroom,Wheat,[Decrease the target's Stamina regeneration by...,3029
17724,Poison of Damage Stamina Regen,3,434,Frost Mirriam,Purple Mountain Flower,Wheat,[Decrease the target's Stamina regeneration by...,1302
2618,Poison of Paralysis,2,428,Ashen Grass Pod,Briar Heart,Human Flesh,"[You are 24% harder to detect for 60 seconds.,...",856
17032,Potion of Regenerate Magicka,2,374,Fire Salts,Moon Sugar,Snowberries,[Magicka regenerates 30% faster for 300 second...,748
10009,Potion of Fortify Pickpocket,2,347,Charred Skeever Hide,Orange Dartwing,Slaughterfish Egg,"[Pickpocketing is 24% easier for 60 seconds., ...",694
22745,Potion of Regenerate Magicka,2,324,Jazbay Grapes,Red Mountain Flower,Taproot,[Magicka is increased by 24 points for 60 seco...,648
13698,Potion of Fortify Two-handed,2,301,Dragon's Tongue,Emperor Parasol Moss,Void Salts,"[Causes 18 points of poison damage., Magicka i...",602


In [20]:
print(f"Optimal Objective Value: {int(m.objVal)}")

Optimal Objective Value: 61143


In [21]:
optimal_df.loc[11281].descriptions

["Decrease the target's Magicka regeneration by 100% for 30 seconds.",
 "Drain the target's Stamina by 6 points per second for 10 seconds.",
 'Resist 6% of magic for 60 seconds.',
 'Can breathe underwater for 359 seconds.']